In [54]:
import numpy as np
# For creating SVG images of states
import cairo

# State definition
class State:
    def __init__(self, board_size, tiles=None):
        # Initialise board size
        self.board_size = board_size
        self.tiles = tiles
        # Initialise rectangles list
        if tiles is None:
            # rectangles are of form (x,y,w,h)
            self.tiles = np.array([[0, 0, board_size, board_size]])
        # Generate list of state areas
        self.areas = self.get_areas()
        # Generate score
        self.score = self.get_mondrian_score()
        # Generate unique state string (for comparing to other states)
        self.state_string = self.get_state_string()
        # Determine if self is legal
        self.is_legal = self.is_legal_state()

    def get_areas(self):
        return self.tiles[:,3] * self.tiles[:,2]

    def get_mondrian_score(self):
        if len(self.areas) == 1:
            return self.areas[0]
        return np.ptp(self.areas)

    def is_legal_state(self):
        for i, t in enumerate(self.tiles):
            for t2 in self.tiles[i+1:]:
                # Check if rectangles are congruent
                if np.all(t[2:] == t2[2:]) or np.all(t[2:] == t2[2:][::-1]):
                    return False
        return True
    
    # Given a rectangle, return a list of its 4 corners as (x,y) tuples
    def get_rect_corner(self, r):
        # Bottom left, bottom right, top left, top right
        return [(r[0], r[1]), (r[0] + r[2], r[1]), (r[0], r[1]+r[3]), (r[0]+r[2], r[1]+r[3])]
    
    def get_corners(self, i,j):
        # Get locations of 4 corners of each rectangle
        c1 = self.get_rect_corner(self.tiles[i,:])
        c2 = self.get_rect_corner(self.tiles[j,:])
        valid_corners = [(0,1), (2,3), (1,0), (3,2), (2,0), (3,1), (1,3), (0,2)]
        # Get list of matching corners in form (i,j) where i, j are corner types
        # 0 is bottom left, 1 is bottom right, 2 is top left, 3 is top right
        corners = [(i,j) for i in range(len(c1)) for j in range(len(c2)) if c1[i] == c2[j]]
        
        # Full edge merge
        if len(corners) == 2:
            return corners[0]
        # The smaller rectangle (consumer) must have the shortest distance in the axis of the shared edge
        if len(corners) == 1:
            if (corners[0] in valid_corners[:4] and self.tiles[i,3] < self.tiles[j,3]):
                return corners[0]
            if (corners[0] in valid_corners[4:] and self.tiles[i,2] < self.tiles[j,2]):
                return corners[0]
        return None
    
    def get_state_string(self):
        # String of sorted area list
        return str(np.sort(self.areas))
    
    def merge_rectangles(self, i, j, c):

        ri = self.tiles[i,:]
        rj = self.tiles[j,:]
        
        # Define changes to the two rectangles position and area based on which corner is matched
        corner_map = {
            (0,1): ((-1*rj[2], 0, rj[2],0), (0, ri[3], 0, -1*ri[3])), 
            (2,3): ((-1*rj[2], 0, rj[2],0), (0, 0, 0, -1*ri[3])), 
            (1,0): ((0, 0, rj[2],0), (0, ri[3], 0, -1*ri[3])), 
            (3,2): ((0, 0, rj[2],0), (0, 0, 0, -1*ri[3])), 
            (2,0): ((0, 0, 0, rj[3]), (ri[2], 0, -1*ri[2], 0)), 
            (3,1): ((0, 0, 0, rj[3]), (0, 0, -1*ri[2], 0)), 
            (1,3): ((0, -1*rj[3], 0, rj[3]), (0, 0, -1*ri[2], 0)), 
            (0,2): ((0, -1*rj[3], 0, rj[3]), (ri[2], 0, -1*ri[2], 0))
        }
        # Apply changes to rectangles
        di,dj = corner_map[c]
        ri = ri + np.array(di)
        rj = rj + np.array(dj)
        
        
        if i < j:
            # Don't add rectangles with 0 area
            if rj[2] == 0 or rj[3] == 0:
                return State(self.board_size, np.vstack([self.tiles[:i,:], ri, self.tiles[i+1:j,:], self.tiles[j+1:,:]]))
            return State(self.board_size, np.vstack([self.tiles[:i,:], ri, self.tiles[i+1:j,:], rj, self.tiles[j+1:,:]]))
        # Don't add rectangles with 0 area
        if rj[2] == 0 or rj[3] == 0:
            return State(self.board_size, np.vstack([self.tiles[:j,:], self.tiles[j+1:i,:], ri, self.tiles[i+1:,:]]))
        return State(self.board_size, np.vstack([self.tiles[:j,:], rj, self.tiles[j+1:i,:], ri, self.tiles[i+1:,:]]))

    # Manual merging, for generating diagrams   
    def merge(self, i, j):
        return self.merge_rectangles(i, j, self.get_corners(i,j))

    def get_all_merges(self):
        merges=[]
        area_order = np.argsort(self.areas)
        # Loop through rectangles from smallest to largest area, comparing only to larger rectangles
        for i in range(len(area_order)):
            for j in range(i+1, len(area_order)):
                # If they share a valid corner, generate the merge
                corner = self.get_corners(area_order[i],area_order[j])
                if corner:
                    merges.append(self.merge_rectangles(area_order[i],area_order[j],corner))
        return merges

    def get_all_legal_merges(self):
        return [s_prime for s_prime in self.get_all_merges() if s_prime.is_legal]

    def box_cut(self, x1, y1, x2, y2):
        
        i = np.argmax(self.areas)
        r = self.tiles[i,:]
        # Top, left, bottom, right and middle rectangles
        new_rects = [(r[0], r[1], r[2], x1),
                    (r[0], r[1]+x1, y1, r[3]-x1),
                    (r[0]+y1, r[1]+x2, r[2]-y1, r[3]-x2),
                    (r[0]+y2, r[1]+x1, r[2]-y2, x2-x1),
                    (r[0]+y1, r[1]+x1, y2-y1, x2-x1)]
        # Add all rectangles in list with area > 0, removing original rectangle r
        return State(self.board_size, np.vstack([self.tiles[:i,:], [rect for rect in new_rects if not rect[2] == 0 and not rect[3] == 0], self.tiles[i+1:,:]]))

    def get_all_box_cuts(self):
        actions = []
        i = np.argmax(self.areas)
        r = self.tiles[i,:]
        # Generate all box cuts with constainrts x1 < x2 <= height(r), y1 < y2 <= width(r) 
        for x2 in range(r[3]+1):
            for y2 in range(r[2]+1):
                for x1 in range(x2):
                    for y1 in range(y2):
                        if x2 == r[3] and y2 == r[2]:
                            break
                        if x2 == 0 and y2 == 0:
                            continue
                        actions.append(self.box_cut(x1, y1, x2, y2))
        return actions
    
    def get_all_legal_box_cuts(self):
        return [b for b in self.get_all_box_cuts() if b.is_legal]
    
    def get_all_actions(self, box_cuts=True):
        return self.get_all_merges() + self.get_all_box_cuts()

    def get_all_legal_actions(self, box_cuts=True):
        return self.get_all_legal_merges() + self.get_all_legal_box_cuts()

In [55]:
# Beam search (modified best first search) algorithm to search for best Mondrian tilings
def SolveMondrian(a, M):
    # Initialise problem
    initial_state = State(a)
    best_state = initial_state
    
    # Set maximum search depth (hyperparameter)
    max_depth=10
    
    # Initialise state memory
    seen = set()
    seen.add(initial_state.state_string)

    # Create beam
    beam = [initial_state]

    for i in range(max_depth):
        # If we have exhausted all states, exit
        if len(beam) == 0:
            return best_state
        new_actions = []
        for s in beam:
            # Add all next legal actions
            for s_prime in s.get_all_legal_actions():
                # Remove non-unique states
                if s_prime.state_string in seen:
                    continue
                seen.add(s_prime.state_string)
                new_actions.append(s_prime)
                # Update best score
                if s_prime.score < best_state.score:
                    best_state = s_prime
        # Prune next exploration to best M solutions at depth i
        beam = sorted(new_actions, key=lambda x: x.score)[:M]
    return best_state

In [56]:
# Variation to allow invalid states for one step (only invalid states generated by valid states)
def SolveMondrian_invalid(a, M):
    # Initialise problem
    initial_state = State(a)
    best_state = initial_state
    
    # Set maximum search depth (hyperparameter)
    max_depth=10
    
    # Initialise Memoization
    seen = set()
    seen.add(initial_state.state_string)

    # Create beam
    beam = [initial_state]

    for i in range(max_depth):
        # If we have exhausted all states, exit
        if len(beam) == 0:
            return best_state
        new_actions = []
        for s in beam:
            # Add all next legal actions
            for s_prime in s.get_all_actions():
                # Remove non-unique states
                if s_prime.state_string in seen:
                    continue
                seen.add(s_prime.state_string)
                # Only allow 1 illegal step (don't let illegal states produce new illegal states)
                if s_prime.is_legal or s.is_legal:
                    new_actions.append(s_prime)
                # Update best score
                if s_prime.is_legal and s_prime.score < best_state.score:
                    best_state = s_prime
        # Prune next exploration to best M solutions at depth i
        beam = sorted(new_actions, key=lambda x: x.score)[:M]
    return best_state

In [57]:
# Generate images for a state
# Parameters: filename (name of output file), M (beam size), s (state object), sf (scaling factor from distances to pixels)
def generate_svg(filename, M, s, sf=50):
    # Ensure minimum surface length so text doesn't cut off
    surface_length = max(450, (s.board_size+3)*sf)

    with cairo.SVGSurface(f"{filename}.svg", surface_length, surface_length) as surface:
        # Generate top text
        cr = cairo.Context(surface)
        cr.set_font_size(18)
        cr.move_to(sf, sf)
        cr.show_text(f"Board Size: {s.board_size}x{s.board_size}    M={M}    Mondrian score: {s.score}")
        cr.set_font_size(13)
        cr.set_line_width(2)
        rects = []

        for i, t in enumerate(s.tiles):
            # Generate random tile colour
            x,y,z = np.random.uniform(0.5, 1.0, size=3)
            cr.set_source_rgb(x, y, z)
            # Get all rectangle sizes (and pad by 1 on left, 2 on top)
            rect = [(t[0]+1)*sf, (t[1]+2)*sf, t[2]*sf, t[3]*sf]
            rects.append(rect)
            # Draw rectangle in random colour
            cr.rectangle(*rect)
            cr.fill()
            # Write rectangle area and shape
            cr.move_to(sf*((t[0]+0.5*t[2])+0.9), sf*((t[1]+0.5*t[3])+1.8))
            cr.set_source_rgb(0, 0, 0)
            cr.show_text(str(t[2]* t[3]))
            cr.move_to(sf*((t[0]+0.5*t[2])+0.9), sf*((t[1]+0.5*t[3])+2.2))
            cr.show_text(str(t[2]) + "x" + str(t[3]))
        # Give all rectangles black outline   
        for r in rects:
            cr.rectangle(*r)
        cr.stroke()
    print("Image Saved")

In [50]:
# Experiment with different a and M values
a_vals = [8, 12, 16, 20]
m_vals = [1, 10, 25, 50]

for a in a_vals:
    print(f"Size {a}:")
    for m in m_vals:
        solution = SolveMondrian(a, m)
        print(f"M={m}: Score={solution.score}")
        generate_svg(f"imgs/size_{a}_M_{m}", m, solution)

Size 8:
M=1: Score=10
Image Saved
M=10: Score=6
Image Saved
M=25: Score=6
Image Saved
M=50: Score=6
Image Saved
Size 12:
M=1: Score=8
Image Saved
M=10: Score=8
Image Saved
M=25: Score=8
Image Saved
M=50: Score=8
Image Saved
Size 16:
M=1: Score=18
Image Saved
M=10: Score=15
Image Saved
M=25: Score=12
Image Saved
M=50: Score=11
Image Saved
Size 20:
M=1: Score=13
Image Saved
M=10: Score=13
Image Saved
M=25: Score=13
Image Saved
M=50: Score=13
Image Saved


In [51]:
# Generate solutions for a=8 to a=20
M = 25
for a in range(8, 21):
    solution = SolveMondrian(a, M)
    print(f"Size {a}: Score = {solution.score}")
    generate_svg(f"imgs/size_{a}_M_{M}", M, solution)

Size 8: Score = 6
Image Saved
Size 9: Score = 6
Image Saved
Size 10: Score = 9
Image Saved
Size 11: Score = 8
Image Saved
Size 12: Score = 8
Image Saved
Size 13: Score = 10
Image Saved
Size 14: Score = 9
Image Saved
Size 15: Score = 8
Image Saved
Size 16: Score = 12
Image Saved
Size 17: Score = 12
Image Saved
Size 18: Score = 12
Image Saved
Size 19: Score = 16
Image Saved
Size 20: Score = 13
Image Saved
